In [1]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20

#cpptools.setup_nlopt(folder='cppfuncs/', do_print = True) #install nlopt

# Solve the model

In [2]:

specs = {}
T = 20



#specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'threads':threads,'bargaining':2}}})
#specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK':False , 'threads':threads,'bargaining':1}}})

#specs.update({f'model_FC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True, 'threads':threads,'bargaining':0}}})
#specs.update({f'model_LC_doHK':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': True , 'threads':threads,'bargaining':1}}}) #ser ikke godt ud!
#specs.update({f'model_NC_doHK':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': True, 'threads':threads,'bargaining':2}}})

print(specs)



# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models.spec = spec

    compile_now = True if do_compile & (m==0) else False
    models.link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models.solve() 
    
    #Save the data 
    T = models.par.T
    print(' saving...')
    models.sim.init_love[:] = 0.2
    models.sim.init_A[:] = 2.0 
    np.random.seed(models.par.seed)
    data1 = Estimate.create_data_new(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
    print(f'Couple {np.mean(models.sim.couple,0)}')
    print(f'HKw {np.mean(models.sim.Kw,0)}')
    print(f'HKm {np.mean(models.sim.Km,0)}')
    print(f'Laborw {np.mean(models.sim.labor_w,0)}')
    print(f'Laborm {np.mean(models.sim.labor_m,0)}')
    print(f'Asset {np.nanmean(models.sim.A,0)}')

    var_list = ('labor_w', 'labor_m', 'cons_w', 'cons_m', 'Vw', 'Vm')
    t_list = (1,9)
    i_A = 250
    i_HK = 1
    i_HKw = i_HK
    i_HKm = i_HK
    i_Z = 0
    i_Zw = i_Z
    i_Zm = i_Z
    i_P = 5
    i_L = 5
    figure.figure_single(models, var_list, t_list, i_A, i_Z, i_HK, model_name = name)
    figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'couple', model_name = name)
    figure.figure_couple(models, var_list, t_list, i_A, i_Zw, i_Zm, i_HKw, i_HKm, i_L, i_P, sol_type = 'remain_couple', model_name = name)



    



#model = models['model FC, NO_HK ']
#model.sim.init_love[:] =0.2
##model.simulate()
#print('HK')*
#print(f'Couple {np.mean(model.sim.couple,0)}')

#print(f'Laborw {np.mean(model.sim.labor_w,0)}')
#print(f'Laborm {np.mean(model.sim.labor_m,0)}')
#print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
#print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
#print(f'Asset {np.nanmean(model.sim.A,0)}')
#print(f'HKw {np.mean(model.sim.Kw,0)}')
#print(f'HKm {np.mean(model.sim.Km,0)}')
#temp = model.sim.power
#I = model.sim.couple<1
#nan = np.zeros(I.shape)
#nan[I] = np.nan
#temp = np.nanmean(temp + nan,axis=0)
#print(f'Power {temp}')


{'model_FC_doHK': {'latexname': 'limited', 'par': {'T': 20, 'do_HK': True, 'threads': 20, 'bargaining': 0}}}
model_FC_doHK loading... solving...
CPU times: total: 1d 5h 27min 38s
Wall time: 3h 5min 47s
 saving...
Couple [1.     0.8612 0.765  0.6822 0.6066 0.5504 0.4988 0.45   0.409  0.3648
 0.3278 0.296  0.2656 0.2446 0.224  0.2118 0.1998 0.1922 0.1888 0.1886]
HKw [0.10114007 0.20114982 0.29798156 0.39176853 0.48125188 0.56782315
 0.65009698 0.72942892 0.8064257  0.87983908 0.94887127 1.01539889
 1.0822262  1.14252285 1.19896535 1.25312687 1.29764341 1.3356394
 1.36689736 1.38794145]
HKm [0.09950293 0.19975698 0.29757416 0.39203322 0.48182455 0.56876318
 0.65237686 0.73248732 0.80744645 0.87762471 0.94465331 1.01125888
 1.07650872 1.13933302 1.19603915 1.24888613 1.29731687 1.33743834
 1.36954863 1.39163406]
Laborw [0.93555395 0.95125784 0.95957856 0.96862227 0.96925804 0.9703783
 0.9722302  0.97400585 0.97407176 0.9745037  0.9746289  0.97432796
 0.97416001 0.97249262 0.96014638 0.9281

In [3]:


T = models.par.T
print(' saving...')
models.sim.init_love[:] = 0.2
    
models.sim.init_A[:] = 2.0 
np.random.seed(models.par.seed)
data1 = Estimate.create_data_new(models,start_p = 1, end_p = T-1, to_xl = True, name_xl = name)
print(f'Couple {np.mean(models.sim.couple,0)}')
print(f'HKw {np.mean(models.sim.Kw,0)}')
print(f'HKm {np.mean(models.sim.Km,0)}')
print(f'Laborw {np.mean(models.sim.labor_w,0)}')
print(f'Laborm {np.mean(models.sim.labor_m,0)}')
print(f'Asset {np.nanmean(models.sim.A,0)}')


model = models

sol = models.sol
par = models.par
sim =  models.sim

print(f'HKw max {np.max(model.sim.Kw,0)}')
print(f'HKm max {np.max(model.sim.Km,0)}')


print(f'HKw min {np.min(model.sim.Kw,0)}')
print(f'HKm min {np.min(model.sim.Km,0)}')
#print(sim.draw_Kw_temp)
print(sim.draw_Kw)

print(f'A max {np.nanmax(model.sim.A,0)}')
print(f'A min {np.nanmin(model.sim.A,0)}')
print(sim.draw_Kw_temp)

print(f'Love max {np.nanmax(model.sim.love,0)}')
print(f'Love mean {np.nanmean(model.sim.love,0)}')
print(f'Love min {np.nanmin(model.sim.love ,0)}')




print(f'laborW max {np.nanmax(model.sim.labor_w,0)}')
print(f'laborW mean {np.nanmean(model.sim.labor_w,0)}')
print(f'LaborW min {np.nanmin(model.sim.labor_w,0)}')

print(par.grid_A)

 saving...
Couple [1.     0.8612 0.765  0.6822 0.6066 0.5504 0.4988 0.45   0.409  0.3648
 0.3278 0.296  0.2656 0.2446 0.224  0.2118 0.1998 0.1922 0.1888 0.1886]
HKw [0.10114007 0.20114982 0.29798156 0.39176853 0.48125188 0.56782315
 0.65009698 0.72942892 0.8064257  0.87983908 0.94887127 1.01539889
 1.0822262  1.14252285 1.19896535 1.25312687 1.29764341 1.3356394
 1.36689736 1.38794145]
HKm [0.09950293 0.19975698 0.29757416 0.39203322 0.48182455 0.56876318
 0.65237686 0.73248732 0.80744645 0.87762471 0.94465331 1.01125888
 1.07650872 1.13933302 1.19603915 1.24888613 1.29731687 1.33743834
 1.36954863 1.39163406]
Laborw [0.93555395 0.95125784 0.95957856 0.96862227 0.96925804 0.9703783
 0.9722302  0.97400585 0.97407176 0.9745037  0.9746289  0.97432796
 0.97416001 0.97249262 0.96014638 0.92817791 0.89083629 0.85073022
 0.80836551 0.73594161]
Laborm [0.93610455 0.95222341 0.95893842 0.96428748 0.96821903 0.96992343
 0.97193416 0.97267448 0.9726235  0.97330358 0.9731902  0.97407483
 0.9738819

In [4]:
stop 

NameError: name 'stop' is not defined

# Estimate the model

In [ ]:
#Save the data 
model = models['model NC, NO_HK ']
model.sim.init_love[:] =0.1
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data1 = Estimate.create_data_new(model,start_p = 9, end_p = 19, to_xl = True, name_xl = 'simulated_data_NC_NO_HK4')


In [ ]:
#Save the data 
#model = models['model LC, NO_HK ']
#model.sim.init_love[:] =0.1
#model.sim.init_A[:] = 6.0
#np.random.seed(model.par.seed)
#data1 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_LC_NO_HK2')


model = models['model LC, HK ']
model.sim.init_love[:] =0.3
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data2 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_LC_HK')

#model = models['model NC, NO_HK ']
#model.sim.init_love[:] =0.1
#model.sim.init_A[:] = 6.0
#np.random.seed(model.par.seed)
#data1 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_NC_NO_HK2')


model = models['model NC, HK ']
model.sim.init_love[:] =0.3
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data2 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_NC_HK')

#Save the data 
#model = models['model FC, NO_HK ']
#model.sim.init_love[:] =0.1
#model.sim.init_A[:] = 6.0
#np.random.seed(model.par.seed)
#data1 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_FC_NO_HK2')


model = models['model FC, HK ']
model.sim.init_love[:] =0.3
model.sim.init_A[:] = 6.0
np.random.seed(model.par.seed)
data2 = Estimate.create_data_new(model,start_p = 2, end_p = 19, to_xl = True, name_xl = 'simulated_data_FC_HK')


In [ ]:

#Estimate the model one time
data_reg = Estimate.aux_est(data2,print_reg=True)
data_reg, Wald_FC, Wald_NC = Estimate.main_est(data_reg,print_reg=True) 